# Clasificación binaria de barcos mediante una CNN en AI::MXNet 
Ships in Satellite Imagery
https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery/data

Marcelo Contreras Kohl, Julio-2025

## 0.- Estructura del dataset
Descargar el dataset  y cear el siguiente árbol de directorios

### Estructura del dataset
MiDataset/ <br>
 ├── shipsnet/ <br>
     ├── clase1/ <br>
     │    ├── img001.jpg <br>
     │    ├── img002.jpg <br>
     │    └── ... <br>
     ├── clase2/ <br>
     │    ├── img101.jpg <br>
     │    ├── img102.jpg <br>
     │    └── ... <br>
     └── claseN/ <br>
          ├── imgN01.jpg <br>
          ├── imgN02.jpg <br>
          └── ... <br>

### Variables de entorno
Establecer variables de entorno para utilizar hilos que controlan paralelización y ejecución en CPU. 

In [1]:
$ENV{MXNET_CPU_WORKER_NTHREADS} = 6;
$ENV{MXNET_ENGINE_TYPE} = 'ThreadedEnginePerDevice';

ThreadedEnginePerDevice


## Módulos requeridos 

In [2]:
use strict;
use warnings;

use AI::MXNet qw(mx gluon);
use AI::MXNet::NDArray qw(nd);
use Scalar::Util qw(blessed);
use List::Util qw(shuffle);

use FindBin;
use lib "$FindBin::Bin/./lib"; # Asegura que los módulos .pm locales se encuentren

use ImageFolderDataset;
use ImageFolderDatasetSubset;
use DataLoader;

# 1.- Preparar DataSet

### Identificar clases y contar

In [3]:
use File::Find;

my $root = 'MiDataset/shipsnet';
my %count = ();

find(
    sub {
        return unless -f && /\.(jpg|png)$/i;
        my $label = (split('/', $File::Find::dir))[-1];
        $count{$label}++;
    },
    $root
);

# 

# Identificar la clase minoritaria
my ($minor_class, $major_class) = sort { $count{$a} <=> $count{$b} } keys %count;

print "\nClase minoritaria: $minor_class ($count{$minor_class})\n";
print "Clase mayoritaria:  $major_class ($count{$major_class})";


Clase minoritaria: class_1 (1000)
Clase mayoritaria:  class_0 (3000)

1

### Histograma

In [4]:
IPerl->load_plugin('Chart::Plotly');

# Crear datos para el histograma a partir de %count
my @clases = sort keys %count;
my @frecuencias = @count{@clases};

my $trace = {
    x => \@clases,
    y => \@frecuencias,
    type => 'bar',
    marker => { color => 'skyblue' },
    name => 'Frecuencia por clase',
};

my $layout = {
    title => 'Distribución de imágenes por clase',
    xaxis => { title => 'Clase' },
    yaxis => { title => 'Cantidad de imágenes' },
    bargap => 0.1,
    width => 400,
    height => 330,
    margin => { l => 50, r => 50, b => 50, t => 50, pad => 0, },
};

my $plot = Chart::Plotly::Plot->new(
    traces => [$trace],
    layout => $layout
);


## Helpers para dividir el dataset y transfomer
### Usar módulo "ImageFolderDataset.pm" para cargar imágenes desde una carpeta y las etiqueta automáticamente según el subdirectorio (clase):

In [5]:

sub split_dataset {
    my ($dataset, %ratios) = @_;

    my $random_state = delete $ratios{random_state};
    srand($random_state) if defined $random_state;
    use AI::MXNet qw(mx);
    mx->random->seed($random_state);

    # Asignar valores por defecto si no están definidos
    $ratios{train} = 0.7  unless defined $ratios{train};
    $ratios{val}   = 0.15 unless defined $ratios{val};
    $ratios{test}  = 0.15 unless defined $ratios{test};

    my $total = $dataset->len;
    my @indices = shuffle(0 .. $total - 1);

    my $train_end = int($ratios{train} * $total);
    my $val_end   = $train_end + int($ratios{val} * $total);

    my @train_indices = @indices[0 .. $train_end - 1];
    my @val_indices   = @indices[$train_end .. $val_end - 1];
    my @test_indices  = @indices[$val_end .. $#indices];

    my $subset = sub {
        my @idx = @_;
        my (@subset_data);
        for my $i (@idx) {
            my ($img, $lbl, $path) = $dataset->at($i);
            push @subset_data, { img => $img, label => $lbl, path => $path };
        }

        return ImageFolderDatasetSubset->new(\@subset_data);
    };

    return (
        train => $subset->(@train_indices),
        validation => $subset->(@val_indices),
        test => $subset->(@test_indices),
    );
}

# Tamaño de la imagen (definido una sola vez)
use vars qw($image_size);  # Para usar en otra celda (Jupyter)
$image_size = 80;          # Si cambias el dataset, solo modificas aquí para usar en redimensionar la imagen

# Transformer (ejemplo: normalización básica)
sub transformer {
    my ($img, $label) = @_;
    $img = mx->image->imresize($img, $image_size, $image_size);  # Esto no cambia nada si ya es 80x80
    $img = $img->astype('float32') / 255;
    #$img = mx->nd->transpose($img, [2, 0, 1]);   # Asegura CHW (3,128,128)
    return ($img, $label);
}

80

  ## Cargar dataset completo desde el directorio y luego dividir (split_dataset) 
  - Split DataSet : 70% entrenamiento, 15% validación, 15% prueba

In [6]:
# Cargar dataset completo 
my $full_dataset = ImageFolderDataset->new('MiDataset/shipsnet', \&transformer);

# Dividir dataset: 
my %datasets = split_dataset($full_dataset, train => 0.7, val => 0.15, test => 0.15, random_state => 42);

print "Train: ", $datasets{train}->len, "\n";
print "Val:   ", $datasets{validation}->len, "\n";
print "Test:  ", $datasets{test}->len, "\n";

print "Dataset cargado y dividido exitosamente.\n";
print "Dataset dividido en ". keys(%datasets), " subconjuntos: ", join(", ", keys %datasets), "\n";

# use Data::Dumper;
# print Dumper($datasets{train}{data}[0]);


Train: 2800
Val:   600
Test:  600
Dataset cargado y dividido exitosamente.
Dataset dividido en 3 subconjuntos: train, test, validation


[21:00:14] /Users/marcont/mxnet_190/src/engine/engine.cc:54: MXNet start using engine: ThreadedEnginePerDevice


1

## Verifcar inconsistencias en Subset train, test, validation (opcional)


In [7]:
my $M_img = 5;

for my $subset (qw(train validation test)) {
    my $dataset = $datasets{$subset};
    print "\nPrimeros $M_img ejemplos del subconjunto '$subset':\n";

    for my $i (0 .. $M_img) {
        my ($img, $label, $path) = $dataset->at($i);
        unless (defined $img && defined $label) {
            warn "Ejemplo $i indefinido\n";
            next;
        }
        print "Ejemplo $i: Etiqueta = $label | Ruta = $path\n";
        print "Dimensiones: ", join("x", @{$img->shape}), "\n";
    }
}

print "--------------------------\n";

# Verificar consistencia de etiquetas en los subconjuntos
print "Verificando consistencia de etiquetas en los subconjuntos...\n";

use File::Spec;

my $label_map = $full_dataset->{label_map};
my $inconsistencias_globales = 0;

for my $subset (qw(train validation test)) {
    my $dataset = $datasets{$subset};
    print "\n📂 Primeros $M_img ejemplos del subconjunto '$subset':\n";

    my $errores = 0;

    my $N = ($datasets{$subset}->len) - 1;
    print "Total ejemplos en '$subset': $N - 1\n";

    for my $i (0 .. $N) {
        my ($img, $label, $path) = $dataset->at($i);
        unless (defined $img && defined $label && defined $path) {
            warn "⚠️  Ejemplo $i indefinido o incompleto\n";
            next;
        }

        print "Ejemplo $i: Etiqueta = $label | Ruta = $path\n";
        print "Dimensiones: ", join("x", @{$img->shape}), "\n";

        # Extraer nombre del subdirectorio que representa la clase
        my @dirs = File::Spec->splitdir($path);
        my $dir_label = $dirs[-2];  # se asume ruta tipo .../class_0/image.png

        if (exists $label_map->{$dir_label}) {
            my $expected = $label_map->{$dir_label};
            if ($label != $expected) {
                warn "❌ Inconsistencia: etiqueta=$label vs esperada=$expected según ruta '$dir_label'\n";
                $errores++;
            }
        } else {
            warn "⚠️  No se reconoce la clase '$dir_label' en label_map\n";
            $errores++;
        }
    }

    if ($errores == 0) {
        print "✅ Sin errores de consistencia en subset '$subset'";
    } else {
        $inconsistencias_globales += $errores;
    }
}

print "\n\n";


if ($inconsistencias_globales == 0) {
    print "\n🎉 ✅ Todos los subconjuntos ('train', 'validation', 'test') están consistentes: sin errores de etiquetas vs directorios.";
} else {
    print "\n⚠️  Se detectaron $inconsistencias_globales inconsistencias en los subconjuntos. Revisar detalles arriba.";
}


Primeros 5 ejemplos del subconjunto 'train':
Ejemplo 0: Etiqueta = 0 | Ruta = MiDataset/shipsnet/class_0/0__20170604_180820_0f52__-122.3545386336167_37.76035896311274.png
Dimensiones: 80x80x3
Ejemplo 1: Etiqueta = 1 | Ruta = MiDataset/shipsnet/class_1/1__20170620_180742_102f__-122.33746157633949_37.73898418986875.png
Dimensiones: 80x80x3
Ejemplo 2: Etiqueta = 0 | Ruta = MiDataset/shipsnet/class_0/0__20170719_180901_101d__-122.32359106663287_37.71792277861482.png
Dimensiones: 80x80x3
Ejemplo 3: Etiqueta = 0 | Ruta = MiDataset/shipsnet/class_0/0__20170709_181333_0e0e__-122.33918938210937_37.76676333587583.png
Dimensiones: 80x80x3
Ejemplo 4: Etiqueta = 0 | Ruta = MiDataset/shipsnet/class_0/0__20180710_180333_1002__-118.20848400499767_33.706847186312324.png
Dimensiones: 80x80x3
Ejemplo 5: Etiqueta = 1 | Ruta = MiDataset/shipsnet/class_1/1__20180205_181811_1030__-122.36586298001781_37.762367278018836.png
Dimensiones: 80x80x3

Primeros 5 ejemplos del subconjunto 'validation':
Ejemplo 0: Eti

1

### Crear subdirectorios y copiar imágenes del dataset dividido al disco 

In [8]:
use strict;
use warnings;
use File::Path qw(make_path);
use File::Copy;
use File::Basename qw(fileparse);
use List::Util qw(shuffle);
use feature 'say';

# --- Crear directorios para split
for my $split (qw(train validation test)) {
    for my $class (qw(class_0 class_1)) {
        make_path("MiDataset/shipsnet_split/$split/$class");
    }
}

# --- Copiar imágenes a sus nuevos directorios
foreach my $split (qw(train validation test)) {

    my $subset = $datasets{$split};

    print "Copiando imágenes al split '$split'...\n";

    for my $item (@{$subset->{data}}) {
        my $src = $item->{path};
        #print "Copiando $src...\n";
        my $label = $item->{label} // (($src =~ /class_0/) ? 0 : 1);
        my $class = ($label == 0) ? 'class_0' : 'class_1';
        my ($basename) = fileparse($src);
        my $dst = "MiDataset/shipsnet_split/$split/$class/$basename";
        copy($src, $dst) or warn "No se pudo copiar $src a $dst: $!";
    }
}




Copiando imágenes al split 'train'...
Copiando imágenes al split 'validation'...
Copiando imágenes al split 'test'...


### Contar las clases en los conjuntos: train, val y test

In [9]:
sub contar_clases {
    my ($dataset) = @_;
    my %conteo;
    for my $i (0 .. $dataset->len - 1) {
        my (undef, $label) = $dataset->at($i);
        $conteo{$label}++;
    }
    return %conteo;
}

my %clases_train = contar_clases($datasets{train});
print "Train: ", join(", ", map { "Clase $_: $clases_train{$_}" } sort keys %clases_train), "\n";

my $n_train = 0;
$n_train += $_ for values %clases_train;
print "Total de imágenes en entrenamiento: $n_train\n";

foreach my $subset (qw(train validation test)) {
    my %cuenta = contar_clases($datasets{$subset});
    print "\nDistribución en '$subset':\n";
    foreach my $clase (sort keys %cuenta) {
        print "  Clase $clase: $cuenta{$clase} imágenes\n";
    }
}

Train: Clase 0: 2102, Clase 1: 698
Total de imágenes en entrenamiento: 2800

Distribución en 'train':
  Clase 0: 2102 imágenes
  Clase 1: 698 imágenes

Distribución en 'validation':
  Clase 0: 450 imágenes
  Clase 1: 150 imágenes

Distribución en 'test':
  Clase 0: 448 imágenes
  Clase 1: 152 imágenes


## Balancear clase solo para entrenamiento

Augmentación: Iterar sobre los archivos de 'train/class_1', aplicar transformaciones y guardar como "aug_XXX_*.png"
Las transformaciones en este celda son: 
- rotación en 90°
- brillo en los tres canales rgb
- ruido en rgb <br>

In [10]:

use Imager;
use File::Glob ':glob';
use File::Basename;
use feature 'say';
use Imager::Color;
use List::Util qw(min max);

use vars qw($root %count);

my $class0_dir = "$root\_split/train/class_0";
my $class1_dir = "$root\_split/train/class_1";
print "Clase 0: $class0_dir\nClase 1: $class1_dir\n";

my @imgs0 = bsd_glob("$class0_dir/*.{jpg,png}", GLOB_BRACE);
my @imgs1 = bsd_glob("$class1_dir/*.{jpg,png}", GLOB_BRACE);

my $n0 = scalar @imgs0;
my $n1 = scalar @imgs1;
my $diff = $n0 - $n1;
print "Clase 0: $n0\nClase 1: $n1\n";

if ($diff <= 0) {
    print "🟢 Ya está balanceado o class_1 es mayor\n";
    exit;
}
print "🔄 Generar $diff nuevas imágenes en class_1...\n";

# Generar augmentaciones distribuidas entre las imágenes originales
my $n_imgs     = scalar @imgs1;
my $counter    = 0;
my $generated  = 0;

for my $i (0 .. $#imgs1) {
    last if $generated >= $diff;

    my $img = Imager->new();
    $img->read(file => $imgs1[$i]) or die $img->errstr;
    my $base = basename($imgs1[$i]);
    $base =~ s/\.(jpg|png)$//i;

    # Llamar a la nueva función "augmentar" que devuelve 3 imágenes transformadas
    my @aug_imgs = augmentar($img);

    for my $aug_img (@aug_imgs) {
        last if $generated >= $diff;

        my $aug_filename = sprintf("%s/aug_%03d_%s.png", $class1_dir, $counter, $base);
        $aug_img->write(file => $aug_filename) or die $aug_img->errstr;
        $counter++;
        $generated++;
        #say "Generada imagen: $aug_filename";
    }
}

print "✅ Se generaron $counter nuevas imágenes augmentadas.\n";


sub augmentar {
    my ($img) = @_;

    # Rotación
    my $rotated = $img->rotate(right => 90);

    # Brillo
    my $bright = $img->copy;
    for my $y (0 .. $bright->getheight - 1) {
        my @scanline = $bright->getscanline(y => $y);
        for my $pixel (@scanline) {
            my $r = List::Util::min(255, int($pixel->red   * 1.2));
            my $g = List::Util::min(255, int($pixel->green * 1.2));
            my $b = List::Util::min(255, int($pixel->blue  * 1.2));
            $pixel = Imager::Color->new($r, $g, $b);
        }
        $bright->setscanline(y => $y, pixels => \@scanline);
    }

    # Ruido
    my $noisy = $img->copy;
    for my $y (0 .. $noisy->getheight - 1) {
        my @scanline = $noisy->getscanline(y => $y);
        for my $pixel (@scanline) {
            my $r = List::Util::max(0, List::Util::min(255, $pixel->red   + int(rand(20) - 10)));
            my $g = List::Util::max(0, List::Util::min(255, $pixel->green + int(rand(20) - 10)));
            my $b = List::Util::max(0, List::Util::min(255, $pixel->blue  + int(rand(20) - 10)));
            $pixel = Imager::Color->new($r, $g, $b);
        }
        $noisy->setscanline(y => $y, pixels => \@scanline);
    }

    return ($rotated, $bright, $noisy);
}


say "✅ Dataset reorganizado y listo para entrenamiento con aumento en class_1";

Clase 0: MiDataset/shipsnet_split/train/class_0
Clase 1: MiDataset/shipsnet_split/train/class_1
Clase 0: 2102
Clase 1: 698
🔄 Generar 1404 nuevas imágenes en class_1...
✅ Se generaron 1404 nuevas imágenes augmentadas.
✅ Dataset reorganizado y listo para entrenamiento con aumento en class_1


1

## 2.- Cargar subdatasets particionados y balanceados en disco

MiDataset/shipsnet_split/
├── train/
│   ├── class_0/
│   └── class_1/
├── validation/
│   ├── class_0/
│   └── class_1/
└── test/
    ├── class_0/
    └── class_1/

In [11]:
use ImageFolderDataset;

print "Cargando datasets desde '$root\_split'...\n";

my $train_dataset = ImageFolderDataset->new("$root\_split/train", \&transformer);
my $val_dataset   = ImageFolderDataset->new("$root\_split/validation", \&transformer);
my $test_dataset  = ImageFolderDataset->new("$root\_split/test", \&transformer);

Cargando datasets desde 'MiDataset/shipsnet_split'...


ImageFolderDataset=HASH(0x7fcae8fc1ac0)

## Preparar DataLoader y hacer batching

In [12]:
use DataLoader;

my $batch_size = 32;

my $train_loader = DataLoader->new(dataset => $train_dataset, batch_size => $batch_size, shuffle => 1);
my $val_loader   = DataLoader->new(dataset => $val_dataset, batch_size => $batch_size, shuffle => 0);
my $test_loader  = DataLoader->new(dataset => $test_dataset, batch_size => $batch_size, shuffle => 0);

DataLoader=HASH(0x7fca9d234ca0)

### Verificar que los labels coincidan con la carpeta

In [13]:
use Imager;
use File::Glob ':glob';
use File::Basename;
use feature 'say';

my $aug_dir = "MiDataset/shipsnet_split/train/class_1";
my @aug_imgs = bsd_glob("$aug_dir/aug_*.png");
my @errores;

say "🔍 Verificando ", scalar(@aug_imgs), " imágenes augmentadas en class_1...";

foreach my $img_path (@aug_imgs) {
    my $img = Imager->new;
    unless ($img->read(file => $img_path)) {
        push @errores, $img_path;
        warn "❌ No se pudo leer: $img_path → ", $img->errstr, "\n";
        next;
    }

    my ($w, $h, $c) = ($img->getwidth, $img->getheight, $img->getchannels);
    unless ($w == 80 && $h == 80 && $c == 3) {
        push @errores, $img_path;
        warn "⚠️ Dimensiones inválidas: $img_path → ${w}x${h} canales=$c\n";
    }
}

if (!@errores) {
    say "✅ Todas las imágenes augmentadas son válidas y tienen tamaño 80x80x3.";
} else {
    say "❌ Se encontraron errores en ", scalar(@errores), " archivos:";
    say $_ for @errores;
}

🔍 Verificando 1404 imágenes augmentadas en class_1...
✅ Todas las imágenes augmentadas son válidas y tienen tamaño 80x80x3.


1

###  Verficar una celda de prueba (Opcional)

In [14]:
my ($data, $label) = $train_loader->next;
print "Batch shape [batch_size, height, width, channels]: ", join("x", @{$data->shape}), "\n";
#print "Labels: ", join(", ", @$label), "\n" if ref($label) eq 'ARRAY';
my $labels = $label->aspdl->unpdl;
$labels = [$labels] unless ref($labels) eq 'ARRAY';
print "Labels: ", join(", ", @$labels), "\n";

Batch shape [batch_size, height, width, channels]: 32x80x80x3
Labels: 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0


1

### Obtener un batch del loader para verificar (Opcional)
Guarda las imágenes en disco (solo visualización) y se renderizan en la celda con html

In [15]:
use PDL;
use PDL::Core ':Func';
use PDL::IO::GD qw(write_true_png);
use IPerl;
use feature 'say';

# Obtener un batch del loader
my ($data, $label, $paths) = $train_loader->next;

# Convertir labels a array Perl
my $labels = $label->aspdl->unpdl;
$labels = [$labels] unless ref($labels) eq 'ARRAY';

# Visualización HTML
my $html = "<div style='display: flex; flex-wrap: wrap;'>\n";

for my $i (0 .. $#$labels) {
    # Convertir imagen a PDL
    my $img = $data->at($i)->aspdl->mv(0, 2);  # CHW → HWC
    $img *= 255 if $img->max <= 1;
    $img = byte($img);

    my $true_label = $labels->[$i];
    my $path = $paths->[$i];
    my ($basename) = $path =~ m{([^/]+\.png)$};

    # Guardar imagen con nombre informativo
    my $filename = sprintf("batch_train_%02d_label_%d_%s", $i, $true_label, $basename);
    write_true_png($img, $filename);

    $html .= "<div style='margin: 5px; text-align: center; font-size: 11px; max-width: 80px;'>
                <img src='$filename' width='80' style='border: 1px solid #ccc;'><br>
                <b>Label:</b> $true_label<br>
                <small style='word-break: break-all;'>$basename</small>
              </div>\n";
}

$html .= "</div>";
IPerl->html($html);

Label: 0 
 0__20171119_181612_0f52__-122.34940708073549_37.76684722887759.png 
 
 
 
 Label: 1 
 aug_1332_1__20171023_175305_1039__-118.1167284926536_33.688419843356876.png 
 
 
 
 Label: 1 
 aug_284_1__20170502_180546_1044__-122.34684178124445_37.76031655598638.png 
 
 
 
 Label: 1 
 aug_979_1__20170717_180818_1010__-122.35291186187388_37.75161631965994.png 
 
 
 
 Label: 1 
 aug_909_1__20170709_181333_0e0e__-122.33155954792313_37.747846835478015.png 
 
 
 
 Label: 1 
 aug_706_1__20170620_180742_102f__-122.34340267518871_37.72939138145052.png 
 
 
 
 Label: 0 
 0__20170419_180519_1009__-122.16279554989674_37.79008541547046.png 
 
 
 
 Label: 0 
 0__20180206_181844_0f34__-122.47874184329346_37.819784433763736.png 
 
 
 
 Label: 0 
 0__20170713_180939_1002__-122.23881862139378_37.7260855587726.png 
 
 
 
 Label: 1 
 aug_768_1__20170702_181120_103a__-122.32438659736084_37.72027977816535.png 
 
 
 
 Label: 1 
 aug_1288_1__20170923_181240_0f42__-122.33785767749494_37.7671344111478.png 
 
 
 
 Label: 1 
 aug_1084_1__20170730_181043_103d__-122.3479088163598_37.74739763610506.png 
 
 
 
 Label: 0 
 0__20170328_181231_0e0f__-122.34837179935694_37.737314484146694.png 
 
 
 
 Label: 1 
 1__20170707_174130_0c0b__-122.33884252598018_37.767464412618764.png 
 
 
 
 Label: 1 
 1__20171023_175304_1039__-118.16090423887094_33.733872379270466.png 
 
 
 
 Label: 1 
 aug_462_1__20170609_180756_103a__-122.34784341495181_37.76648707436548.png 
 
 
 
 Label: 0 
 0__20170712_181004_0f12__-122.24949974932905_37.70665862436784.png 
 
 
 
 Label: 0 
 0__20180706_180314_1024__-118.22376638803574_33.73897403989648.png 
 
 
 
 Label: 0 
 0__20170501_180553_1018__-122.13414121169063_37.7024068602389.png 
 
 
 
 Label: 0 
 0__20171119_181612_0f52__-122.32920768462218_37.73677435185076.png 
 
 
 
 Label: 0 
 0__20170718_180714_102f__-122.21365198865877_37.64961493369818.png 
 
 
 
 Label: 0 
 0__20161218_180844_0e26__-122.25570705681058_37.844651758189976.png 
 
 
 
 Label: 0 
 0__20170703_180945_1009__-122.34270476653322_37.71976142405217.png 
 
 
 
 Label: 1 
 1__20170909_181730_0e0f__-122.32821888700255_37.736930957319274.png 
 
 
 
 Label: 1 
 aug_127_1__20170106_180851_0e30__-122.34661289674644_37.759233712714554.png 
 
 
 
 Label: 0 
 0__20180711_180349_1042__-118.09005815577095_33.777970019609.png 
 
 
 
 Label: 1 
 aug_665_1__20170619_180826_1032__-122.34445250403449_37.70297957681386.png 
 
 
 
 Label: 0 
 0__20171127_185539_0f29__-122.2742442285676_37.85279800227779.png 
 
 
 
 Label: 0 
 0__20171021_181323_1003__-122.11289798802032_37.73729170506815.png 
 
 
 
 Label: 1 
 aug_1352_1__20171023_184815_0f21__-118.21694562064921_33.73755340598831.png 
 
 
 
 Label: 1 
 aug_391_1__20170522_180635_0f42__-122.3492190437419_37.74943283414725.png 
 
 
 
 Label: 0 
 0__20180712_211332_0f06__-118.22382768245986_33.6634069085091.png

Warning: Prototype mismatch: sub main::max (@) vs none at (eval 1037) line 6.

Prototype mismatch: sub main::min (@) vs none at (eval 1037) line 6.


## 3.- Denición del Modelo:  red neuronal convolucional (CNN).

In [16]:
use AI::MXNet qw(mx);
use AI::MXNet::Gluon::NN qw(nn);

# 1. Contexto CPU (o GPU si quieres)
my $ctx = mx->cpu;

# 2. Definir la Red CNN
my $net = nn->Sequential();
$net->add(
    # Primera capa convolucional: Conv2D + BatchNorm + MaxPool
    nn->Conv2D(channels => 64, kernel_size => [3, 3], activation => 'relu', padding => [1,1],
               weight_initializer => mx->init->Xavier(), 
               bias_initializer => 'zeros'),
    nn->BatchNorm(),
    nn->MaxPool2D(pool_size => [2, 2]),

    # Segunda capa convolucional
    nn->Conv2D(channels => 32, kernel_size => [3, 3], activation => 'relu', padding => [1,1],
               weight_initializer => mx->init->Xavier(),
               bias_initializer => 'zeros'),
    nn->BatchNorm(),
    nn->MaxPool2D(pool_size => [2, 2]),

    # Tercera capa convolucional
    nn->Conv2D(channels => 16, kernel_size => [3, 3], activation => 'relu', padding => [1,1],
               weight_initializer => mx->init->Xavier(),
               bias_initializer => 'zeros'),
    nn->BatchNorm(),
    nn->MaxPool2D(pool_size => [2, 2]),

    # Aplanar + Dense + Dropout
    #nn->GlobalAvgPool2D(),  # convierte cualquier tamaño a (1,1,C)
    nn->Flatten(),

    nn->Dense(64, activation => 'relu'),
    nn->Dropout(0.5),

    nn->Dense(32, activation => 'relu'),
    nn->Dropout(0.5),

    # Capa de salida (2 neuronas para clasificación binaria con SoftmaxCrossEntropyLoss)
    nn->Dense(2)
);

# 3. Inicializar pesos
$net->initialize(mx->init->Xavier(), ctx => $ctx);


# 4. Activar pesos diferidos con un forward pass
# ===============================================================
# NOTA IMPORTANTE:
# Las imágenes se cargan inicialmente en formato NHWC (Batch, Height, Width, Channels).
# MXNet espera las entradas en formato NCHW (Batch, Channels, Height, Width).
# Aquí verificamos que la imagen tiene las dimensiones [Batch, 80, 80, 3]
# para luego transponerla correctamente a [Batch, 3, 80, 80].
#
# Si no se transpone bien, la red falla con errores de tamaño en MaxPool2D.
# Este paso asegura que la red procese correctamente imágenes 80x80 RGB.
# ===============================================================

# 4. Activar pesos diferidos con un forward pass
my ($data, $label) = $train_loader->next;
if ($data->shape->[1] == $image_size && $data->shape->[3] == 3) {
    $data = $data->transpose([0, 3, 1, 2]);  # NHWC -> NCHW
}
$data  = $data->as_in_context($ctx);
$label = $label->as_in_context($ctx);
my $output = $net->($data);  # ← activa pesos diferidos

# 5. Optimizer
my $trainer = mx->gluon->Trainer(
    $net->collect_params(),
    'adam',
    { learning_rate => 0.001, wd => 0.001 }
);

# 6. Función de pérdida
my $loss_fn = AI::MXNet::Gluon::Loss::SoftmaxCrossEntropyLoss->new();

# 7. Ver red
print $net;

Sequential(
  (0): Conv2D(64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
  (1): BatchNorm(momentum=0.9, axis=1, fix_gamma=0, eps=1e-05)
  (2): MaxPool2D(size=(2,2), stride=(2,2), padding=(0,0), ceil_mode=0)
  (3): Conv2D(32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
  (4): BatchNorm(fix_gamma=0, eps=1e-05, axis=1, momentum=0.9)
  (5): MaxPool2D(size=(2,2), stride=(2,2), padding=(0,0), ceil_mode=0)
  (6): Conv2D(16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
  (7): BatchNorm(axis=1, momentum=0.9, eps=1e-05, fix_gamma=0)
  (8): MaxPool2D(size=(2,2), stride=(2,2), padding=(0,0), ceil_mode=0)
  (9): Flatten
  (10): Dense(64 -> 0, Activation(relu))
  (11): Dropout(p = 0.5)
  (12): Dense(32 -> 0, Activation(relu))
  (13): Dropout(p = 0.5)
  (14): Dense(2 -> 0, linear)
)

1

## Mostrar pesos y bias

In [17]:
for my $i (0 .. $#{$net}) {
    my $layer = ${$net}[$i];
    print "[$i] ", ref($layer), "\n";

    # Verifica y muestra pesos
    if ($layer->can('weight') and defined $layer->weight) {
        my $w_data = eval { $layer->weight->data($ctx) };
        if ($w_data) {
            my $shape_w = join(",", @{ $w_data->shape });
            print "    Pesos: [$shape_w]\n";
        }
    }

    # Verifica y muestra bias
    if ($layer->can('bias') and defined $layer->bias) {
        my $b_data = eval { $layer->bias->data($ctx) };
        if ($b_data) {
            my $shape_b = join(",", @{ $b_data->shape });
            print "    Bias: [$shape_b]\n";
        }
    }
}

[0] AI::MXNet::Gluon::NN::Conv2D
    Pesos: [64,3,3,3]
    Bias: [64]
[1] AI::MXNet::Gluon::NN::BatchNorm
[2] AI::MXNet::Gluon::NN::MaxPool2D
[3] AI::MXNet::Gluon::NN::Conv2D
    Pesos: [32,64,3,3]
    Bias: [32]
[4] AI::MXNet::Gluon::NN::BatchNorm
[5] AI::MXNet::Gluon::NN::MaxPool2D
[6] AI::MXNet::Gluon::NN::Conv2D
    Pesos: [16,32,3,3]
    Bias: [16]
[7] AI::MXNet::Gluon::NN::BatchNorm
[8] AI::MXNet::Gluon::NN::MaxPool2D
[9] AI::MXNet::Gluon::NN::Flatten
[10] AI::MXNet::Gluon::NN::Dense
    Pesos: [64,1600]
    Bias: [64]
[11] AI::MXNet::Gluon::NN::Dropout
[12] AI::MXNet::Gluon::NN::Dense
    Pesos: [32,64]
    Bias: [32]
[13] AI::MXNet::Gluon::NN::Dropout
[14] AI::MXNet::Gluon::NN::Dense
    Pesos: [2,32]
    Bias: [2]


## 4.- Entrenamiento y validación 

In [18]:

# --- Función auxiliar F1-score por batch ---
sub f1_score {
    my ($preds, $labels) = @_;
    my $pred_classes = $preds->argmax(axis => 1);
    my $tp = (($pred_classes * $labels)->sum)->asscalar;
    my $fp = ((($pred_classes == 1) * ($labels == 0))->sum)->asscalar;
    my $fn = ((($pred_classes == 0) * ($labels == 1))->sum)->asscalar;
    my $precision = $tp + $fp > 0 ? $tp / ($tp + $fp) : 0;
    my $recall    = $tp + $fn > 0 ? $tp / ($tp + $fn) : 0;
    my $f1 = $precision + $recall > 0 ? 2 * ($precision * $recall) / ($precision + $recall) : 0;
    return $f1;
}

# --- F1 macro para la época completa ---
sub f1_score_macro {
    my ($outputs, $labels) = @_;
    my $preds = $outputs->argmax(axis => 1);
    my $pred_array  = $preds->aspdl;
    my $label_array = $labels->aspdl;
    my ($tp0, $fp0, $fn0, $tp1, $fp1, $fn1) = (0, 0, 0, 0, 0, 0);

    for my $i (0 .. $label_array->dim(0) - 1) {
        my $true = $label_array->at($i);
        my $pred = $pred_array->at($i);
        if ($true == 0) {
            $tp0++ if $pred == 0;
            $fp1++ if $pred == 1;
            $fn0++ if $pred == 1;
        } else {
            $tp1++ if $pred == 1;
            $fp0++ if $pred == 0;
            $fn1++ if $pred == 0;
        }
    }

    my $f1_0 = (2 * $tp0) / (2 * $tp0 + $fp0 + $fn0 + 1e-10);
    my $f1_1 = (2 * $tp1) / (2 * $tp1 + $fp1 + $fn1 + 1e-10);
    return ($f1_0 + $f1_1) / 2;
}

# --- Entrenamiento por época ---
sub train_one_epoch {
    my ($epoch, $net, $train_loader, $trainer, $loss_fn, $ctx) = @_;

    $net->train;

    my $metric = mx->metric->Accuracy();
    $metric->reset();

    my ($train_loss, $num_batches) = (0, 0);

    $train_loader->reset();
    print "\nIniciando época $epoch...\n";


    while (my ($data, $label) = $train_loader->next) {
        if ($data->shape->[1] == $image_size && $data->shape->[3] == 3) {
            $data = $data->transpose([0, 3, 1, 2]);  # NHWC → NCHW
        }
        $data  = $data->as_in_context($ctx);
        $label = $label->as_in_context($ctx);

        my $loss;
        my $output;
      
        autograd->record(sub {
            # Paso 1: calcular el loss normal
            $output = $net->($data);
            $loss = $loss_fn->($output, $label);

            if ($num_batches == 0) {
                #print "Ejemplo pérdida: ", $loss->aspdl, "\n";
                #print "Pérdida ponderada (media): ", $loss->mean->asscalar, "\n";

                my $probs = $output->softmax;
                my $preds = $probs->argmax(axis => 1);

                #print "== Primer batch de validación ==\n";
                #print "Probabilidades:\n", $probs->aspdl, "\n";
                #print "Predicciones:\n", $preds->aspdl, "\n";
                #print "Etiquetas verdaderas:\n", $label->aspdl, "\n";
            }

            $loss->backward;
        });

        $trainer->step($data->shape->[0]);  # tamaño del batch

        $train_loss += $loss->mean->asscalar;
        $metric->update([$label], [$output]);
        $num_batches++;
    }

    my ($name, $train_acc) = $metric->get();
    my $avg_train_loss = $train_loss / $num_batches;

    return ($avg_train_loss, $train_acc);
}




# --- Validación por época ---
sub validate_one_epoch {
    my ($net, $val_loader, $loss_fn, $ctx, $epoch) = @_;
    $net->eval;

    my ($val_loss, $total, $f1_sum, $batches) = (0, 0, 0, 0);
    my $val_metric = mx->metric->Accuracy();
    $val_metric->reset();
    $val_loader->reset();

    my @all_outputs;
    my @all_labels;

    while (my ($data, $label) = $val_loader->next) {
        if ($data->shape->[1] == $image_size && $data->shape->[3] == 3) {
            $data = $data->transpose([0, 3, 1, 2]);
        }
        $data  = $data->as_in_context($ctx);
        $label = $label->as_in_context($ctx);

        my $output = $net->($data);
        die "ERROR: output es undef\n" unless defined $output;

        my $loss = $loss_fn->($output, $label);
        $val_loss += $loss->mean->asscalar;
        $total    += $label->shape->[0];

        
        my $f1 = f1_score($output, $label);
        $f1_sum += $f1;

        $val_metric->update([$label], [$output]); 
        push @all_outputs, $output;
       
        push @all_labels,  $label;
        $batches++;

        my ($pred_label, $true_label) = ($output->argmax(axis => 1), $label);
            #print "Batch pred: ", $pred_label->aspdl, "\n";
            #print "Batch true: ", $true_label->aspdl, "\n";
    }

    my $avg_val_loss = $val_loss / $total;
    my $avg_val_f1   = $f1_sum / $batches;
    my (undef, $val_acc) = $val_metric->get();

    my $f1_macro = 'N/A';
    if (@all_outputs && @all_labels) {
        my $concat_output = AI::MXNet::NDArray->concat(@all_outputs, { dim => 0 });
        my $concat_label  = AI::MXNet::NDArray->concat(@all_labels,  { dim => 0 });
        $f1_macro = f1_score_macro($concat_output, $concat_label);
    }

    return ($avg_val_loss, $f1_macro, $val_acc);
}


#======================================================
# --- Bucle principal de entrenamiento + validación ---
#======================================================
use Time::HiRes qw(time);
print "Iniciando entrenamiento...\n";
print time(), " - ", scalar(localtime(time())), "\n";


my @acc_train_hist = ();
my @loss_train_hist = ();
my @acc_val_hist = ();
my @loss_val_hist = ();
my @f1_val_hist = ();


# calcular actualizaciones por época
my $updates_per_epoch = int($n_train / $batch_size);  # 2800 / 32 = 87
print "Actualizaciones por época: $updates_per_epoch\n";

use AI::MXNet::LRScheduler;
# --- Scheduler: reduce el LR en cada época ---
my $lr_scheduler = AI::MXNet::FactorScheduler->new(
    step    => $updates_per_epoch * 5,  # cada N_updates actualizaciones (no épocas)
    factor => 0.5, # el learning rate disminuye un 10% en cada “step” definido. 0.5 es mas agresivo   0.95 es ma conservador
    # base_lr => 0.001, # valor típico cuando se usa optimizador "adam"
    # stop_factor_lr => 1e-6
);
print ref($lr_scheduler),"\n";  # Debe ser 'AI::MXNet::FactorScheduler'

# --- optimizador manualmente ---
my $optimizer = AI::MXNet::Optimizer->create(
    'adam',
    learning_rate => 0.001,
    lr_scheduler  => $lr_scheduler  # 👈 esto sigue funcionando
);


# --- Trainer con scheduler ---
my $trainer = mx->gluon->Trainer(
    $net->collect_params(),
    $optimizer
);

# --- Inicializar historial de learning rate ---
my @lr_history = ();

#--- Variables de control EarlyStoping ---
my $best_val_loss = 1e8;  # 1e10 es algo muy alto
my $patience = 5;
my $patience_counter = 0;

# --- Número de épocas ---
my $epochs = 50;
print "Número de épocas: $epochs\n";

for my $epoch (1 .. $epochs) {

    print "\n⏳ Época $epoch\n";

    # --- Entrenamiento ---
    my ($train_loss, $train_acc) = train_one_epoch($epoch, $net, $train_loader, $trainer, $loss_fn, $ctx);
    printf("🟢 Entrenamiento: pérdida=%.4f, acc=%.2f%%\n", $train_loss, $train_acc*100);

    # Obtener el learning rate actual después de cada época
    my $num_update = $epoch * $updates_per_epoch;
    my $current_lr = $lr_scheduler->($num_update);
    push @lr_history, $current_lr;


    # --- Validación ---
    my ($val_loss, $val_f1, $val_acc) = validate_one_epoch($net, $val_loader, $loss_fn, $ctx, $epoch);
    printf("🔵 Validación: pérdida=%.4f, F1=%.4f, acc=%.2f%%\n", $val_loss, $val_f1, $val_acc*100);

    # --- Monitorea el learning rate ---
    # my $lr_actual = $trainer->learning_rate; # error: Too few arguments for method learning_rate (expected 2, got 1)
    my $lr_actual = $trainer->{optimizer}->{learning_rate};

    print "Epoch $epoch - Learning Rate actual: $lr_actual\n";


    # --- Early Stopping ---
    if ($val_loss < $best_val_loss) {
        print "Mejoró val_loss: $val_loss (anterior: $best_val_loss)\n";
        $best_val_loss = $val_loss;
        $patience_counter = 0;
        $net->save_parameters('best_model.params');  # Guarda mejores pesos
    } else {
        $patience_counter++;
        print "No mejoró. Paciencia: $patience_counter / $patience\n";

        if ($patience_counter >= $patience) {
            print "Early Stopping activado en época $epoch.\n";
            last;  # Salir del ciclo
        }
    }

    #--- Registro (historia) ---
    push @loss_train_hist, $train_loss;
    push @acc_train_hist,  $train_acc;
    push @loss_val_hist, $val_loss;
    push @acc_val_hist,  $val_acc;
    push @f1_val_hist,   $val_f1;
}

print "\n🎉 Entrenamiento finalizado.\n";
print time(), " - ", scalar(localtime(time())), "\n";


Iniciando entrenamiento...
1764633810.82273 - Mon Dec  1 21:03:30 2025
Actualizaciones por época: 87
AI::MXNet::FactorScheduler
Número de épocas: 50

⏳ Época 1

Iniciando época 1...
🟢 Entrenamiento: pérdida=1.4570, acc=75.02%
🔵 Validación: pérdida=0.0063, F1=0.8979, acc=92.50%
Epoch 1 - Learning Rate actual: 0.001
Mejoró val_loss: 0.0062975678841273 (anterior: 100000000)

⏳ Época 2

Iniciando época 2...
🟢 Entrenamiento: pérdida=0.3709, acc=84.92%
🔵 Validación: pérdida=0.0042, F1=0.9292, acc=94.67%
Epoch 2 - Learning Rate actual: 0.001
Mejoró val_loss: 0.00416699185501784 (anterior: 0.0062975678841273)

⏳ Época 3

Iniciando época 3...
🟢 Entrenamiento: pérdida=0.2583, acc=90.20%
🔵 Validación: pérdida=0.0044, F1=0.9179, acc=94.17%
Epoch 3 - Learning Rate actual: 0.001
No mejoró. Paciencia: 1 / 5

⏳ Época 4

Iniciando época 4...
🟢 Entrenamiento: pérdida=0.2420, acc=91.20%
🔵 Validación: pérdida=0.0032, F1=0.9435, acc=95.67%
Epoch 4 - Learning Rate actual: 0.001
Mejoró val_loss: 0.0032467289

1

Warning: Update[437]: Changed learning rate to 5.00000e-04

Update[872]: Changed learning rate to 2.50000e-04

Update[1307]: Changed learning rate to 1.25000e-04

Update[1742]: Changed learning rate to 6.25000e-05

Update[2177]: Changed learning rate to 3.12500e-05

Update[2612]: Changed learning rate to 1.56250e-05

Update[3047]: Changed learning rate to 7.81250e-06


## Plot learning rate history
### Requiere que @lr_history exista (se llena durante el entrenamiento)

In [19]:
IPerl->load_plugin('Chart::Plotly');
use strict;
use warnings;
use PDL;
use Chart::Plotly;
use Chart::Plotly::Plot;
use Chart::Plotly::Trace::Scatter;


if (!@lr_history) {
    warn "No hay datos en \@lr_history. Ejecuta el entrenamiento primero para registrar el LR.\n";
} else {
    my @epochs = (1 .. scalar @lr_history);
    my @lrs    = map { sprintf "%.6f", $_ } @lr_history;

    my $trace = {
        x    => \@epochs,
        y    => \@lrs,
        # type => 'scatter',
        mode => 'lines',
        name => 'Learning Rate',
        line => { color => 'royalblue', width => 2 },
        marker => { size => 6 },
    };

    my $layout = {
        title => 'Evolución del Learning Rate',
        xaxis => { title => 'Época' },
        yaxis => { title => 'Learning Rate'}, # escala log para visualizar mejor la decay
        width  => 700,
        height => 380,
        margin => { l => 70, r => 30, b => 50, t => 60 },
    };

    my $plot = Chart::Plotly::Plot->new(
        traces => [$trace],
        layout => $layout,
    );

    
}

In [20]:
# IPerl->load_plugin('Chart::Plotly');

# --- Datos de ejemplo ---
my @epochs = (1 .. 30);

my @acc_train = map { sprintf "%.5f", $_ } @acc_train_hist;
my @acc_val = map { sprintf "%.5f", $_ } @acc_val_hist;
my @loss_train = map { sprintf "%.5f", $_ } @loss_train_hist;
my @loss_val = map { sprintf "%.5f", $_ } @loss_val_hist;

# --- Trazas ---
my $trace_acc_train = {
    x => [@epochs], y => [@acc_train], name => 'Precisión Entrenamiento',
    mode => 'lines', line => { color => 'blue' }, type => 'scatter',
};

my $trace_acc_val = {
    x => [@epochs], y => [@acc_val], name => 'Precisión Validación',
    mode => 'lines', line => { color => 'orange' }, type => 'scatter', 
};

my $trace_loss_train ={
    x => [@epochs], y => [@loss_train], name => 'Pérdida Entrenamiento',
    mode => 'lines', line => { color => 'green' }, xaxis => 'x2', yaxis => 'y2', type => 'scatter',    
};

my $trace_loss_val = {
    x => [@epochs], y => [@loss_val], name => 'Pérdida Validación',
    mode => 'lines', line => { color => 'red' }, xaxis => 'x2', yaxis => 'y2', type => 'scatter',    
};

# --- Layout con Subplots ---
 my $layout = {
     title => 'Precisión y Pérdida del Modelo',
     grid => { rows => 1, columns => 2, pattern => 'independent' },
     plot_bgcolor => 'white',
     paper_bgcolor => 'white',
     xaxis => { title => 'Épocas', gridcolor => '#e5ecf6' },
     yaxis => { title => 'Precisión', range => [0.0, 1.0], gridcolor => '#e5ecf6' },
     xaxis2 => { title => 'Épocas', gridcolor => '#e5ecf6' },
     yaxis2 => { title => 'Pérdida', gridcolor => '#e5ecf6' },
     legend => { x => 1.05, y => 1 },
     width => 950,
     height => 450,
 };

# --- Crear Plot ---
my $plot = Chart::Plotly::Plot->new(
    traces => [$trace_acc_train, $trace_acc_val, $trace_loss_train, $trace_loss_val],
    layout => $layout
);



## Reporte de validación

In [21]:
sub final_validation_report {
    my ($net, $val_loader, $loss_fn, $ctx) = @_;

    my (@all_preds, @all_labels);
    my $total_loss = 0;
    $val_loader->reset(); # Recorremos TODOS los datos otra vez desde el inicio.

    while (my ($data, $label) = $val_loader->next) {
        if ($data->shape->[1] == $image_size && $data->shape->[3] == 3) {
            $data = $data->transpose([0, 3, 1, 2]);
        }
        $data  = $data->as_in_context($ctx);
        $label = $label->as_in_context($ctx);

        my $output = $net->($data);
        my $loss   = $loss_fn->($output, $label);
        $total_loss += $loss->mean->asscalar;

        my $preds = $output->argmax(axis => 1);
        push @all_preds,  @{$preds->aspdl->unpdl};
        push @all_labels, @{$label->aspdl->unpdl};
    }

    my $n = scalar @all_labels;
    my ($tp, $tn, $fp, $fn) = (0, 0, 0, 0);
    for my $i (0..$n-1) {
        my $t = $all_labels[$i];
        my $p = $all_preds[$i];
        if    ($t == 1 && $p == 1) { $tp++ }
        elsif ($t == 0 && $p == 0) { $tn++ }
        elsif ($t == 0 && $p == 1) { $fp++ }
        elsif ($t == 1 && $p == 0) { $fn++ }
    }

    # Métricas
    my $acc = ($tp + $tn) / $n;
    my $precision_0 = $tn + $fn ? $tn / ($tn + $fn) : 0;
    my $recall_0    = $tn + $fp ? $tn / ($tn + $fp) : 0;
    my $f1_0        = $precision_0 + $recall_0 ? 2 * $precision_0 * $recall_0 / ($precision_0 + $recall_0) : 0;
    my $support_0   = $tn + $fp;

    my $precision_1 = $tp + $fp ? $tp / ($tp + $fp) : 0;
    my $recall_1    = $tp + $fn ? $tp / ($tp + $fn) : 0;
    my $f1_1        = $precision_1 + $recall_1 ? 2 * $precision_1 * $recall_1 / ($precision_1 + $recall_1) : 0;
    my $support_1   = $tp + $fn;

    my $macro_precision = ($precision_0 + $precision_1) / 2;
    my $macro_recall    = ($recall_0 + $recall_1) / 2;
    my $macro_f1        = ($f1_0 + $f1_1) / 2;

    my $weighted_precision = ($precision_0 * $support_0 + $precision_1 * $support_1) / $n;
    my $weighted_recall    = ($recall_0 * $support_0 + $recall_1 * $support_1) / $n;
    my $weighted_f1        = ($f1_0 * $support_0 + $f1_1 * $support_1) / $n;

    # Mostrar reporte
    printf "\n              precision    recall  f1-score   support\n";
    printf "     class_0     %6.2f     %6.2f     %6.2f     %6d\n", $precision_0, $recall_0, $f1_0, $support_0;
    printf "     class_1     %6.2f     %6.2f     %6.2f     %6d\n", $precision_1, $recall_1, $f1_1, $support_1;
    printf "\n    accuracy                         %6.2f     %6d\n", $acc, $n;
    printf "   macro avg     %6.2f     %6.2f     %6.2f     %6d\n", $macro_precision, $macro_recall, $macro_f1, $n;
    printf "weighted avg     %6.2f     %6.2f     %6.2f     %6d\n\n", $weighted_precision, $weighted_recall, $weighted_f1, $n;

    #my $avg_loss = $total_loss / $val_loader->num_batches;
    #printf "\nAverage Loss (por batch): %.5f\n", $avg_loss;



    return ($tn, $fp, $fn, $tp);
}


my ($tn, $fp, $fn, $tp) = final_validation_report($net, $test_loader, $loss_fn, $ctx);

print "\nMatriz de Confusión:\n";
print "               Pred_0   Pred_1\n";
printf "Real_0      %6d   %6d\n", $tn, $fp;
printf "Real_1      %6d   %6d\n", $fn, $tp;


              precision    recall  f1-score   support
     class_0       0.99       0.98       0.98        448
     class_1       0.94       0.97       0.95        152

    accuracy                           0.98        600
   macro avg       0.97       0.97       0.97        600
weighted avg       0.98       0.98       0.98        600


Matriz de Confusión:
               Pred_0   Pred_1
Real_0         439        9
Real_1           5      147


1

In [22]:
IPerl->load_plugin('Chart::Plotly');

use ConfusionMatrixPlot qw(plot_confusion_matrix);

# --- Matriz de Confusión --- [$tn, $fp], [$fn, $tp]
my $matrix_ref = [
    [$tn, $fp],   # Real_0: Pred_0, Pred_1
    [$fn, $tp],   # Real_1: Pred_0, Pred_1
];
my $x_labels_ref = ['Pred_0', 'Pred_1'];
my $y_labels_ref = ['Real_0', 'Real_1'];

#my $color_scale = 'Hot';  # Puedes cambiar a 'Cividis', 'Hot', etc.

my $plot = plot_confusion_matrix($matrix_ref, $x_labels_ref, $y_labels_ref, 'Matriz de Confusión','Viridis');
$plot;

In [23]:
print "Precisión en el entrenamiento por época:\n";
for my $i (0..$#acc_train_hist) {
    printf("Época %2d: %.4f\n", $i+1, $acc_train_hist[$i]);
}

Precisión en el entrenamiento por época:
Época  1: 0.7502
Época  2: 0.8492
Época  3: 0.9020
Época  4: 0.9120
Época  5: 0.9213
Época  6: 0.9232
Época  7: 0.9436
Época  8: 0.9489
Época  9: 0.9479
Época 10: 0.9508
Época 11: 0.9541
Época 12: 0.9608
Época 13: 0.9584
Época 14: 0.9605
Época 15: 0.9619
Época 16: 0.9660
Época 17: 0.9650
Época 18: 0.9662
Época 19: 0.9667
Época 20: 0.9698
Época 21: 0.9643
Época 22: 0.9691
Época 23: 0.9693


### Ver imágenes de errores

In [24]:
# 2. Preparar referencias:
my $error_indices_ref = [];
my $predicted_labels_ref = [];

# 3. Obtener los errores recorriendo de nuevo:
my $index = 0;
$test_loader->reset();
while (my ($data, $label, $paths) = $test_loader->next) {
    if ($data->shape->[1] == $image_size && $data->shape->[3] == 3) {
        $data = $data->transpose([0, 3, 1, 2]);
    }
    $data  = $data->as_in_context($ctx);
    $label = $label->as_in_context($ctx);

    my $output = $net->($data);
    my $preds = $output->argmax(axis => 1);
    my @preds_arr  = @{$preds->aspdl->unpdl};
    my @labels_arr = @{$label->aspdl->unpdl};

    for my $i (0 .. $#labels_arr) {
        push @$predicted_labels_ref, [$index, $labels_arr[$i], $preds_arr[$i]];

        if ($preds_arr[$i] != $labels_arr[$i]) {
            push @$error_indices_ref, $index;
        }
        $index++;
    }
}

# 4. Verificar que están bien:
print "\nErrores detectados en índices: @$error_indices_ref\n";


Errores detectados en índices: 2 6 80 101 162 231 351 415 416 430 455 526 544 575


1

## Galería de errores

In [25]:
sub generar_galeria_errores {
    my ($dataset, $error_indices_ref, $predicted_labels_ref, $nombre_base) = @_;

    # Crear un hash para buscar rápidamente [real, predicho] por índice
    my %pred_map = map { $_->[0] => [ $_->[1], $_->[2] ] } @$predicted_labels_ref;

    my $html = "<div style='display: flex; flex-wrap: wrap;'>\n";

    for my $i (@$error_indices_ref) {
        my ($img, $label, $path) = $dataset->at($i);
        next unless defined $img;

        my ($real, $predicho) = @{ $pred_map{$i} // [undef, undef] };
        next unless defined $real and defined $predicho;

        # Convertir imagen a PDL
        my $pdl_img = $img->aspdl->mv(0, 2);  # CHW → HWC
        $pdl_img *= 255 if $pdl_img->max <= 1;
        $pdl_img = byte($pdl_img);

        my ($basename) = $path =~ m{([^/]+\.png)$};
        my $filename = sprintf("${nombre_base}_%02d_real_%d_pred_%d.png", $i, $real, $predicho);
        write_true_png($pdl_img, $filename);

        # Mostrar también el path
        $html .= "<div style='margin: 5px; text-align: center; font-size: 11px; max-width: 80px;'>
                    <img src='$filename' width='80' style='border: 1px solid #ccc;'><br>
                    <b>Real:</b> $real<br><b>Pred:</b> $predicho<br>
                    <!--<small>$basename</small><br>-->
                    <small style='word-break: break-all;'>$path</small>
                  </div>\n";
    }

    $html .= "</div>\n";
    IPerl->html($html);
}

generar_galeria_errores($test_loader->{dataset}, $error_indices_ref, $predicted_labels_ref, 'error');

Real: 0 Pred: 1 
 <!-- 0__20161218_180845_0e26__-122.5166757517887_37.77330323976371.png -->
 MiDataset/shipsnet_split/test/class_0/0__20161218_180845_0e26__-122.5166757517887_37.77330323976371.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20170222_231255_0c27__-122.4275778136165_37.65187449612287.png -->
 MiDataset/shipsnet_split/test/class_0/0__20170222_231255_0c27__-122.4275778136165_37.65187449612287.png 
 
 
 
 Real: 1 Pred: 0 
 <!-- 1__20170901_181520_0e14__-122.35176479998303_37.7815966425164.png -->
 MiDataset/shipsnet_split/test/class_1/1__20170901_181520_0e14__-122.35176479998303_37.7815966425164.png 
 
 
 
 Real: 1 Pred: 0 
 <!-- 1__20170716_180816_103a__-122.34916689604478_37.739833546904364.png -->
 MiDataset/shipsnet_split/test/class_1/1__20170716_180816_103a__-122.34916689604478_37.739833546904364.png 
 
 
 
 Real: 1 Pred: 0 
 <!-- 1__20160820_233143_0c53__-122.34848736520453_37.77073581030887.png -->
 MiDataset/shipsnet_split/test/class_1/1__20160820_233143_0c53__-122.34848736520453_37.77073581030887.png 
 
 
 
 Real: 1 Pred: 0 
 <!-- 1__20160820_233143_0c53__-122.34722781164949_37.75902080540278.png -->
 MiDataset/shipsnet_split/test/class_1/1__20160820_233143_0c53__-122.34722781164949_37.75902080540278.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20171127_181539_101e__-122.36377962246702_37.723256986220896.png -->
 MiDataset/shipsnet_split/test/class_0/0__20171127_181539_101e__-122.36377962246702_37.723256986220896.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20180712_211331_0f06__-118.15240457586779_33.73514245946434.png -->
 MiDataset/shipsnet_split/test/class_0/0__20180712_211331_0f06__-118.15240457586779_33.73514245946434.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20171211_181524_1004__-122.04265779755478_37.681771686642676.png -->
 MiDataset/shipsnet_split/test/class_0/0__20171211_181524_1004__-122.04265779755478_37.681771686642676.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20161227_180931_0e30__-122.37070569236482_37.769666826687526.png -->
 MiDataset/shipsnet_split/test/class_0/0__20161227_180931_0e30__-122.37070569236482_37.769666826687526.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20180202_181820_103b__-122.40653507472607_37.80860004330038.png -->
 MiDataset/shipsnet_split/test/class_0/0__20180202_181820_103b__-122.40653507472607_37.80860004330038.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20170822_175102_1006__-118.23303537600333_33.74399529967306.png -->
 MiDataset/shipsnet_split/test/class_0/0__20170822_175102_1006__-118.23303537600333_33.74399529967306.png 
 
 
 
 Real: 0 Pred: 1 
 <!-- 0__20170730_181514_0e20__-122.28516743555205_37.67827239169895.png -->
 MiDataset/shipsnet_split/test/class_0/0__20170730_181514_0e20__-122.28516743555205_37.67827239169895.png 
 
 
 
 Real: 1 Pred: 0 
 <!-- 1__20170901_181520_0e14__-122.35948134600083_37.76776767609284.png -->
 MiDataset/shipsnet_split/test/class_1/1__20170901_181520_0e14__-122.35948134600083_37.76776767609284.png

## Evaluación Final (test_loader)
Bloque para evaluar el modelo ya entrenado en el conjunto de prueba, calcular la precisión (accuracy) y además cómo guardar y cargar el modelo:

In [26]:
my ($correct, $total) = (0, 0);

$test_loader->reset;
while (my ($data, $labels) = $test_loader->next) {
    # Convertir a NCHW si vienen en NHWC
    $data = $data->transpose([0, 3, 1, 2]) if $data->shape->[1] == $image_size;

    my $output = $net->($data);
    my $preds = $output->argmax(axis => 1);

    my $matches = $preds->equal($labels)->sum->asscalar;
    $correct += $matches;
    $total   += $labels->shape->[0];
}

my $acc = $total > 0 ? $correct / $total : 0;
printf("Precisión en conjunto de prueba: %.2f%%\n", 100 * $acc);

Precisión en conjunto de prueba: 97.67%


1

## Guardar el modelo entrenado:

In [27]:
$net->save_parameters('modelo_entrenado_aug_XX.params');
print "Modelo guardado como 'modelo_entrenado_aug_XX.params'\n";

Modelo guardado como 'modelo_entrenado_aug_XX.params'


1